In [1]:
import re
import src.utils as utils
import numpy as np
import tensorflow as tf
import src.benchmark as benchmark
from tqdm.auto import tqdm

In [2]:
# import src.generate_vocabulary as gv
#
# gv.generate_vocabulary(['.\\data\\data'], ['py'], 25000)

In [3]:
# with open('.\\data\\python100k_train.txt') as f:
#     data_dirs = f.readlines()
#
#
# data_dirs = [re.sub(r'/', r'\\', ('.\\data\\'+d)[:-1]) for d in data_dirs]
# data_dirs = np.random.choice(data_dirs, size=50000, replace=False)
#
# tokenized_files = utils.tokenize_data(data_dirs)

In [7]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from src.model import Model

vocab_size = 20001 #CONST
embedding_dim = 32 #CONST
rnn_units = 128
batch_size = 128 #CONST
win_size = 5
model = Model(vocab_size, embedding_dim, rnn_units, batch_size, win_size, '..\\checkpoints\\2\\model.h5', None)
model.prepare_predictions('.\\vocabulary.voc', '..\\checkpoints\\2\\model.h5')
# print(model.get_prediction('', 100))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 32)             640032    
_________________________________________________________________
lstm (LSTM)                  (1, None, 128)            82432     
_________________________________________________________________
dense (Dense)                (1, None, 20001)          2580129   
Total params: 3,302,593
Trainable params: 3,302,593
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
with open('..\\data\\my_eval.txt') as f:
    data_dirs = f.readlines()

np.random.seed(43)
data_dirs = [re.sub(r'/', r'\\', ('..\\data\\'+d)[:-1]) for d in data_dirs]
# data_dirs = np.random.choice(data_dirs, size=9, replace=False)

top_1, top_3, top_5, ranked_5, ranked_10, all_predictions = 0,0,0,0,0,0

In [9]:
progress = tqdm(data_dirs, position=0)

for file in progress:
    result = benchmark.single_benchmark(model, file, html=False)
    top_1 += result[0]
    top_3 += result[1]
    top_5 += result[2]
    ranked_5+= result[3]
    ranked_10 += result[4]
    all_predictions += result[5]

    if all_predictions != 0:
        progress.set_postfix({'top_1' :top_1/all_predictions,
                              'top_3':top_3/all_predictions,
                              'top_5':top_5/all_predictions,
                              'ranked_10':ranked_10/all_predictions,
                              'ranked':ranked_5/all_predictions})
print(f'accuracies: \n\ttop_1: {"%.2f" % ( top_1/all_predictions )}'
      f'\n\ttop_3: {"%.2f" % ( top_3/all_predictions )}'
      f'\n\ttop_5: {"%.2f" % ( top_5/all_predictions )}'
      f'\n\tranked_5: {"%.2f" % (ranked_5/all_predictions )}'
      f'\n\tranked_10: {"%.2f" % (ranked_10/all_predictions )}')





KeyboardInterrupt: 